In [64]:
import psycopg2
import timeit
from timeit import Timer
from sqlalchemy import select, create_engine, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects import postgresql

import load_env
from src.graphql.core.config import settings
import src.graphql.models.recipe_models_autogen as rma

In [41]:
def constructQuery(recipe_id: str):
    stmt = select(rma.Item, rma.RecipeItemGroup.item_inputs_key, rma.ItemGroupItemStacks.item_stacks_stack_size) \
        .join(rma.ItemGroupItemStacks, rma.ItemGroupItemStacks.item_stacks_item_id == rma.Item.id) \
        .join(rma.RecipeItemGroup, rma.RecipeItemGroup.item_inputs_id == rma.ItemGroupItemStacks.item_group_id) \
        .filter(rma.RecipeItemGroup.recipe_id == recipe_id)
    return stmt


def constructQueryWithCompilation(recipe_id: str):
    stmt = constructQuery(recipe_id)
    compiled_query = stmt.compile(
        dialect=postgresql.dialect(),
        compile_kwargs={"literal_binds": True},
    )
    query_str = str(compiled_query).replace(":", "%")
    params = tuple(compiled_query.params.values())
    return query_str, params


def makeSQLASession():
    engine = create_engine(
        f"postgresql://{settings.POSTGRES_USER}:{settings.POSTGRES_PASSWORD}@{settings.POSTGRES_SERVER}:{settings.POSTGRES_PORT}/{settings.POSTGRES_DB}",
        pool_size=30,  # Set the number of connections to maintain in the pool
        max_overflow=0,  # Set the maximum overflow size of the pool
    )
    session_maker = sessionmaker(
        bind=engine,
        expire_on_commit=False,
        autocommit=False,
        autoflush=False,
    )
    session = session_maker()
    return session


def runSQLA(stmt, session):
    result = session.execute(stmt).all()
    return result


def createPsyConnection():
    return psycopg2.connect(
        host=settings.HOST_URL,
        port=settings.POSTGRES_PORT,
        database=settings.POSTGRES_DB,
        user=settings.POSTGRES_USER,
        password=settings.POSTGRES_PASSWORD,
    )


def createPsyCursor(connection):
    return connection.cursor()


def runPsy(cursor, query_str, params):
    cursor.execute(query_str, params)
    return cursor.fetchall()



In [46]:
sqla_timer = Timer('runSQLA(query, session)', """
from __main__ import runSQLA, constructQuery, makeSQLASession;
recipe_id="r~--eEYPHsN5GqWygrEHan4A==";
query=constructQuery(recipe_id);
session=makeSQLASession();
""")
print(f'SQLAlchemy: {sqla_timer.timeit(10000)}')
psy_timer = Timer('runPsy(cursor, query_str, params)', """
from __main__ import runPsy, constructQueryWithCompilation, createPsyConnection, createPsyCursor;
recipe_id="r~--eEYPHsN5GqWygrEHan4A==";
query_str, params=constructQueryWithCompilation(recipe_id);
connection=createPsyConnection();
cursor=createPsyCursor(connection);
""")
print(f'Psycopg2: {psy_timer.timeit(10000)}')

SQLAlchemy: 5.558451937000427
Psycopg2: 4.752735991000009


In [68]:
from src.graphql.scalars.recipe_scalar import NEI_Item

def getQueryStrAndParams(stmt):
    compiled_query = stmt.compile(
        dialect=postgresql.dialect(),
        compile_kwargs={"literal_binds": True},
    )
    query_str = str(compiled_query).replace(":", "%")
    params = tuple(compiled_query.params.values())
    return query_str, params

stmt = select(rma.Item, rma.RecipeItemGroup.item_inputs_key, rma.ItemGroupItemStacks.item_stacks_stack_size) \
    .join(rma.ItemGroupItemStacks, rma.ItemGroupItemStacks.item_stacks_item_id == rma.Item.id) \
    .join(rma.RecipeItemGroup, rma.RecipeItemGroup.item_inputs_id == rma.ItemGroupItemStacks.item_group_id) \
    .filter(rma.RecipeItemGroup.recipe_id == 'r~--eEYPHsN5GqWygrEHan4A==')
query_str, params = getQueryStrAndParams(stmt)
connection = createPsyConnection()
cursor = createPsyCursor(connection)
results = runPsy(cursor, query_str, params)

def constructDictFromORM(
        results,
        orm_object,
        suffix_keys,
        include={},
        exclude=[],
        rename={},
    ):
    column_names = [c_attr.key for c_attr in inspect(orm_object).mapper.column_attrs]
    column_names.extend(suffix_keys)
    
    dictionaries = []
    for result in results:
        d = dict(zip(column_names, result))
        d.update(include)
        for key in exclude:
            d.pop(key)
        for key, value in rename.items():
            d[value] = d.pop(key)

        dictionaries.append(d)
    
    return dictionaries

d = constructDictFromORM(results, rma.Item, ['stack_size', 'position'], {'input': True})
item_inputs = [NEI_Item(**item) for item in d]
print(item_inputs)

[NEI_Item(id='i~gregtech~gt.metaitem.01~23355', item_id=7197, position=1, stack_size=0, image_file_path='item/gregtech/gt.metaitem.01~23355.png', internal_name='gt.metaitem.01', item_damage=23355, localized_name='Magnetic Steel Rod', max_damage=0, max_stack_size=64, mod_id='gregtech', nbt='', tooltip='Magnetic Steel Rod (#7197/23355)\\u000agregtech:gt.metaitem.01\\u000aFe\\u2085\\u2080C', unlocalized_name='gt.metaitem.01.23355', input=True, output_probability=-1.0), NEI_Item(id='i~gregtech~gt.metaitem.01~23019', item_id=7197, position=1, stack_size=1, image_file_path='item/gregtech/gt.metaitem.01~23019.png', internal_name='gt.metaitem.01', item_damage=23019, localized_name='Aluminium Rod', max_damage=0, max_stack_size=64, mod_id='gregtech', nbt='', tooltip='Aluminium Rod (#7197/23019)\\u000agregtech:gt.metaitem.01\\u000aAl', unlocalized_name='gt.metaitem.01.23019', input=True, output_probability=-1.0), NEI_Item(id='i~gregtech~gt.blockmachines~1341', item_id=2086, position=4, stack_size